In [1]:
import pandas as pd
import numpy as np
import csv
import re
file_path = "proj2_data.csv"
row1=None
row2=None
separator = "|"
dec_sep = ","
with open(file_path, newline='') as f:
  reader = csv.reader(f)
  row1 = next(reader)
  row2 = ",".join(next(reader))
  
print(row2)
  
if row1 is not None:
  row1 = row1[0]
  if len(row1.split(separator)) == 1: separator = ";"
  if len(row1.split(separator)) == 1: separator = ","

if separator == ",":
  dec_sep = "."
else:
  row2 = row2.split(separator)
  reg_dot = re.compile(r'\d+\.\d+')
  reg_comma = re.compile(r'\d+,\d+')
  dot_cnt = 0
  comma_cnt = 0
  for col in row2:
    if re.search(reg_dot, col) is not None and len(re.search(reg_dot, col).group()) == len(col):
      dot_cnt+=1
    if re.search(reg_comma, col) is not None and len(re.search(reg_comma, col).group())==len(col):
      comma_cnt+=1
  if dot_cnt > 0 and comma_cnt <= dot_cnt:
    dec_sep = '.'
  else:
    dec_sep = ','

print(dec_sep)
df = pd.read_csv(file_path, sep=separator, decimal=dec_sep)
print(df)
df.to_pickle('./proj2_ex01.pkl')

Rowan Harrington|drones|python|wej|3,1|2,0|4,4|3,16666666666667|dostateczny|3,5 pts|dostateczny
,
           full_name     field   language code  task_1  task_2  task_3  \
0   Rowan Harrington    drones     python  wej     3.1     2.0     4.4   
1         Nash Wyatt    racing       java  sfe     4.2     2.0     2.0   
2     Jadiel Ramirez     media  cplusplus  vaw     4.0     4.9     3.0   
3     Makaila Atkins    racing      swift  ugt     4.1     5.0     4.8   
4     Melanie Fuller    racing     python  owb     2.7     2.0     2.0   
5      Layla Woodard    drones     python  wuf     2.8     2.0     2.0   
6         Edwin Hale     media      swift  ceq     4.9     3.0     3.4   
7       Travis Rocha  robotics  cplusplus  eir     5.0     4.9     5.0   
8       Ricky Howell     media       java  vos     2.1     2.0     2.0   
9    Addyson Brennan    drones       java  cov     4.4     3.7     4.0   
10      Megan Barker    drones      swift  oqq     3.9     4.0     4.4   
11    Abel Hen

In [2]:
scale = {}
with open('./proj2_scale.txt') as f:
  for (idx, row) in enumerate(f):
    scale[row.replace("\n", "")] = idx+1

print(scale)

df_first_copy = df.copy()
print(df_first_copy)

columns_with_scale_values = set()

for column_name, column in df_first_copy.items():
    for idx, value in column.items():
        if value in scale.keys():
            df_first_copy.at[idx, column_name] = scale[value]
            columns_with_scale_values.add(column_name)

for column_name in columns_with_scale_values:
    df_first_copy[column_name] = df_first_copy[column_name].astype(int)
        
print(columns_with_scale_values)

print(df_first_copy)
df_first_copy.to_pickle('./proj2_ex02.pkl')

{'niedostateczny': 1, 'mierny': 2, 'dostateczny': 3, 'dobry': 4, 'bardzo dobry': 5}
           full_name     field   language code  task_1  task_2  task_3  \
0   Rowan Harrington    drones     python  wej     3.1     2.0     4.4   
1         Nash Wyatt    racing       java  sfe     4.2     2.0     2.0   
2     Jadiel Ramirez     media  cplusplus  vaw     4.0     4.9     3.0   
3     Makaila Atkins    racing      swift  ugt     4.1     5.0     4.8   
4     Melanie Fuller    racing     python  owb     2.7     2.0     2.0   
5      Layla Woodard    drones     python  wuf     2.8     2.0     2.0   
6         Edwin Hale     media      swift  ceq     4.9     3.0     3.4   
7       Travis Rocha  robotics  cplusplus  eir     5.0     4.9     5.0   
8       Ricky Howell     media       java  vos     2.1     2.0     2.0   
9    Addyson Brennan    drones       java  cov     4.4     3.7     4.0   
10      Megan Barker    drones      swift  oqq     3.9     4.0     4.4   
11    Abel Hendricks    raci

In [12]:
print(columns_with_scale_values)
df_second_copy = df.copy()

for column in columns_with_scale_values:
    df_second_copy[column] = pd.Categorical(df_second_copy[column], categories=list(scale.keys()))


df_second_copy.to_pickle('./proj2_ex03.pkl')

{'final_grade', 'task_grade'}
['niedostateczny', 'mierny', 'dostateczny', 'dobry', 'bardzo dobry']
full_name        object
field            object
language         object
code             object
task_1          float64
task_2          float64
task_3          float64
tasks_avg       float64
task_grade     category
jury_score       object
final_grade    category
dtype: object


In [14]:
import re

def extract_number(text):
    pattern = r'-{0,1}[0-9]+[\.,]{0,1}[0-9]*'
    match = re.search(pattern, str(text))
    if match:
        number_str = match.group().replace(',', '.')
        return float(number_str)
    else:
        return np.nan

numeric_columns = df.select_dtypes(include=np.number).columns
non_numeric_columns = df.columns.difference(numeric_columns).tolist()
print(non_numeric_columns)

def contains_number(cell_value):
    return bool(re.search(r'-{0,1}[0-9]+[\.,]{0,1}[0-9]*', str(cell_value)))

columns_with_numbers = []

for col in df.columns:
    if col not in numeric_columns:
        if df[col].apply(contains_number).any():
            columns_with_numbers.append(col)
            
new_df = df[columns_with_numbers].copy()
for col in new_df.columns:
  new_df[col] = new_df[col].apply(extract_number)
print(new_df)
new_df.to_pickle('./proj2_ex04.pkl')

['code', 'field', 'final_grade', 'full_name', 'jury_score', 'language', 'task_grade']
    jury_score
0          3.5
1          5.0
2          3.5
3          2.0
4          2.0
5          3.5
6         -1.0
7          0.0
8          NaN
9          NaN
10         2.0
11         4.0


In [40]:
og_df = df[non_numeric_columns].copy()
matching_columns = []
for col in og_df.columns:
  if not og_df[col].isin(scale.keys()).any() and not og_df[col].str.contains(r'[^a-z]').any() and len(og_df[col].unique()) <= 10:
    matching_columns.append(col)

print(matching_columns)
for (idx, col) in enumerate(matching_columns):
  og_df_copy = og_df[col].copy()
  og_df_encoded = pd.get_dummies(og_df_copy)
  og_df_encoded.to_pickle(f"./proj2_ex05_{idx+1}.pkl")
  print(og_df_encoded)

['field', 'language']
    drones  media  racing  robotics
0     True  False   False     False
1    False  False    True     False
2    False   True   False     False
3    False  False    True     False
4    False  False    True     False
5     True  False   False     False
6    False   True   False     False
7    False  False   False      True
8    False   True   False     False
9     True  False   False     False
10    True  False   False     False
11   False  False    True     False
    cplusplus   java  python  swift
0       False  False    True  False
1       False   True   False  False
2        True  False   False  False
3       False  False   False   True
4       False  False    True  False
5       False  False    True  False
6       False  False   False   True
7        True  False   False  False
8       False   True   False  False
9       False   True   False  False
10      False  False   False   True
11      False  False    True  False
